# Vehicle Detection and Tracking
In this project, our goal is to implement a pipeline to identify vehicles in a video from a front-facing camera on a car.

The pipeline is broken into 3 steps
* Build a classifier: to decide whether a square image is a car or not
* Sliding windows technique to search for vehicles in images
* Vehicle tracking: once we have a high confidence vehicle detection, we can eventually estimate where it will appear in each subsequent frame

## Build a car/noncar classifer
### Dataset
We will use the following training dataset
* Labeled data for [vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip) and [non-vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip)
* Additional labeled data from [Udacity](https://github.com/udacity/self-driving-car/tree/master/annotations) which contains data from [CrowdAI](http://bit.ly/udacity-annoations-crowdai) and from [Autti](http://bit.ly/udacity-annotations-autti)